## Part 1

In [1]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=5)

def test(method, input, expected):
    actual = method(input)
    if actual == expected:
        print(f'\t☑ - {method.__name__}({input}) = {expected} = {actual}')
    else:
        print(f'\t☐ - {method.__name__}({input}) = {expected} ≠ {actual}')

In [2]:
import re

def parse_data(input:str) -> tuple[list[tuple[int,int]], list[int]]:
    raw_rules, raw_updates = input.split('\n\n')

    rules = [(int(x), int(y)) for (x,y) in re.findall(r'(\d+)\|(\d+)', raw_rules)]
    updates = []
    
    for raw_update in raw_updates.split():
        updates.append([int(x) for x in re.findall(r'(\d+),?', raw_update)])

    return (rules, updates)

In [3]:
print(parse_data(puzzle.examples[0].input_data))

([(47, 53), (97, 13), (97, 61), (97, 47), (75, 29), (61, 13), (75, 53), (29, 13), (97, 29), (53, 29), (61, 53), (97, 53), (61, 29), (47, 13), (75, 47), (97, 75), (47, 61), (75, 61), (47, 29), (75, 13), (53, 13)], [[75, 47, 61, 53, 29], [97, 61, 53, 29, 13], [75, 29, 13], [75, 97, 47, 61, 53], [61, 13, 29], [97, 13, 75, 29, 47]])


In [4]:
def build_rules(rules_input:list[tuple[int,int]]) -> dict[int, tuple[set[int], set[int]]]:
    rules = {}
    
    for predecessor, successor in rules_input:
        
        if predecessor not in rules:
            rules[predecessor] = (set(), set())

        if successor not in rules:
            rules[successor] = (set(), set())

        rules[predecessor][1].add(successor)
        rules[successor][0].add(predecessor)

    return rules

def _determine_valid_update(rules, update):
    for i, page in enumerate(update):
        predecessors, successors = rules[page]
        
        # Careful this is inverted. I'm checking what follows to make sure it wasn't a require predecessor
        if predecessors & set(update[i+1:]):
            # print(f'Predecessor rule violation for {page} - Rules: {predecessor}')
            valid = False
            break
        if successors & set(update[:i]):
            # print('Successor rule violation')
            valid = False
            break
    else:
        return True

    return False

def find_middle_valid_pages(input:str) -> list[int]:

    raw_rules, raw_updates = parse_data(input)

    rules = build_rules(raw_rules)

    valid_updates = []

    for update in raw_updates:
        if _determine_valid_update(rules, update):
            # print(f'Valid update {update}')
            valid_updates.append(update)
            
    # Assumption here that the list length is not even.
    middle_of_updates = [valid_update[int(len(valid_update)/2)] for valid_update in valid_updates]
    return middle_of_updates


In [5]:
    
print(sum(find_middle_valid_pages(puzzle.examples[0].input_data)))

143


In [6]:
print(sum(find_middle_valid_pages(puzzle.input_data)))

4959


## Part 2

In [7]:
import random

def fix_rules(input):
    raw_rules, raw_updates = parse_data(input)

    rules = build_rules(raw_rules)

    invalid_updates = []

    for update in raw_updates:
        if not _determine_valid_update(rules, update):
            invalid_updates.append(update)

    valid_updates = []

    for invalid_update in invalid_updates:
        valid_found = False
        i = 0
        while i < 10000 and not valid_found:
            i += 1
            random.shuffle(invalid_update)
            if(_determine_valid_update(rules, invalid_update)):
                valid_found = True
                valid_updates.append(invalid_update)

    print(invalid_updates)

    middle_of_updates = [valid_update[int(len(valid_update)/2)] for valid_update in valid_updates]
    return middle_of_updates

def build_rules(rules_input:list[tuple[int,int]]) -> dict[int, tuple[set[int], set[int]]]:
    rules = {}
    
    for predecessor, successor in rules_input:
        
        if predecessor not in rules:
            rules[predecessor] = (set(), set())

        if successor not in rules:
            rules[successor] = (set(), set())

        rules[predecessor][1].add(successor)
        rules[successor][0].add(predecessor)

    return rules


In [8]:

sum(fix_rules(puzzle.examples[0].input_data))

[[97, 75, 47, 61, 53], [61, 29, 13], [97, 75, 47, 29, 13]]


123

In [9]:
sum(fix_rules(puzzle.input_data))

[[37, 11, 68, 16, 59, 69, 85, 15, 62, 83, 39, 71, 34, 28, 43, 33, 17, 75, 58], [48, 81, 33, 15, 34, 75, 73, 71, 24, 31, 16], [68, 24, 33, 57, 31, 71, 16, 74, 81, 34, 48, 69, 19], [62, 22, 75, 16, 39, 28, 11, 15, 19, 59, 37, 85, 81, 24, 69, 68, 73, 57, 33, 31, 48, 43, 34], [56, 57, 19, 86, 85, 28, 75, 81, 59, 11, 39, 37, 31, 68, 74, 34, 32, 16, 24, 62, 69], [83, 73, 12, 17, 54, 18, 89, 42, 92, 22, 53], [32, 28, 18, 57, 24, 19, 42, 11, 39, 34, 37, 48, 56, 86, 85, 75, 43], [37, 71, 11, 57, 16, 83, 58, 33, 13, 17, 62, 92, 43, 77, 69, 15, 68, 22, 75, 73, 98], [24, 28, 48, 19, 73, 11, 81, 15, 37], [17, 82, 71, 15, 22, 92, 69, 94, 75, 77, 54, 52, 73, 53, 12, 89, 83, 58, 33, 57, 11, 29, 98], [85, 68, 57, 37, 81, 71, 31, 12, 33], [47, 32, 74, 89, 31, 39, 24, 19, 42, 81, 53, 48, 85, 16, 59, 26, 86, 18, 87, 94, 56, 82, 34], [59, 33, 75, 68, 39, 34, 85, 81, 37, 62, 73, 28, 31, 57, 16, 19, 71, 22, 15, 83, 24, 69, 43], [74, 77, 86, 26, 87, 56, 12, 52, 29, 38, 53, 54, 42, 48, 19, 92, 94], [26, 37, 56

740

I knew better, but I had to try



In [10]:
def determine_valid_update(rules, update) -> tuple[bool, list[int]]:
    valid_update = True
    invalid_pages = []
    
    for i, page in enumerate(update):
        predecessors, successors = rules[page]
        
        # Careful this is inverted. I'm checking what follows to make sure it wasn't a require predecessor
        if predecessors & set(update[i+1:]):
            # print(f'Predecessor rule violation for {page} - Rules: {predecessor}')
            valid_update = False
            invalid_pages.append(page)
            
        if successors & set(update[:i]):
            # print('Successor rule violation')
            valid_update = False
            invalid_pages.append(page)

    return (valid_update, invalid_pages)

def fix_rules(input):
    raw_rules, raw_updates = parse_data(input)

    rules = build_rules(raw_rules)

    invalid_updates = []

    for update in raw_updates:
        valid, violations = determine_valid_update(rules, update)
        if valid:
            continue
        
        # print(update, violations)
        invalid_updates.append(update)

        for page in update:
            if page not in rules:
                print('Page wasn"t in the rules. I"ll have to be smarter"')

    # print(invalid_updates)

    # valid_updates = []

    # for invalid_update in invalid_updates:
    #     valid_found = False
    #     while i < 10000 and not valid_found:
    #         random.shuffle(invalid_update)
    #         if(_determine_valid_update(rules, invalid_update)):
    #             valid_found = True
    #             valid_updates.append(invalid_update)

    # print(invalid_updates)

    # middle_of_updates = [valid_update[int(len(valid_update)/2)] for valid_update in valid_updates]
    # return middle_of_updates

    return [0]


sum(fix_rules(puzzle.examples[0].input_data))
sum(fix_rules(puzzle.input_data))

0

Eventually I'll learn not to shortcut. Looks like all of the pages are covered by the rules. I can use the rules to build them back up right?


In [11]:
def determine_valid_update(rules, update) -> tuple[bool, list[int]]:
    valid_update = True
    invalid_pages = []
    
    for i, page in enumerate(update):
        predecessors, successors = rules[page]
        
        # Careful this is inverted. I'm checking what follows to make sure it wasn't a require predecessor
        if predecessors & set(update[i+1:]):
            # print(f'Predecessor rule violation for {page} - Rules: {predecessor}')
            valid_update = False
            invalid_pages.append(page)
            
        if successors & set(update[:i]):
            # print('Successor rule violation')
            valid_update = False
            invalid_pages.append(page)

    return (valid_update, invalid_pages)

def fix_rules(input):
    raw_rules, raw_updates = parse_data(input)

    rules = build_rules(raw_rules)

    invalid_updates = []

    for update in raw_updates:
        valid, violations = determine_valid_update(rules, update)
        if valid:
            continue
        
        # print(update, violations)
        invalid_updates.append(update)

        for page in update:
            if page not in rules:
                print('Page wasn"t in the rules. I"ll have to be smarter"')
            

    valid_updates = []

    for update in invalid_updates:
        valid_order = [update[0]]
        for page in update[1:]:
            # print(f"[{page}]: {rules[page]}")
            for i in range(len(valid_order)):
                if page in rules[valid_order[i]][0]:
                    valid_order.insert(i, page)
                    break
            else:
                valid_order.append(page)

        valid_updates.append(valid_order)

    middle_of_updates = [valid_update[int(len(valid_update)/2)] for valid_update in valid_updates]
    return middle_of_updates





In [12]:
sum(fix_rules(puzzle.examples[0].input_data))

123

In [13]:
sum(fix_rules(puzzle.input_data))

4655